In [22]:
import pykeen
from pykeen.pipeline import pipeline
from pykeen.datasets import get_dataset
pykeen.env()

Key,Value
OS,posix
Platform,Linux
Release,6.5.0-41-generic
Time,Wed Jun 19 21:31:42 2024
Python,3.12.4
PyKEEN,1.10.2
PyKEEN Hash,aa183023
PyKEEN Branch,master
PyTorch,2.3.1+cu121
CUDA Available?,true


In [2]:
dataset = get_dataset(dataset="hetionet")

In [34]:
# Run transe model with baseline parameters
def run_pipeline(config: dict):

    pipeline_result = pipeline(
        dataset=config["dataset"],
        dataset_kwargs={
            "random_state": config["seed"],
            "create_inverse_triples": config["train"]["create_inverse"],
        },
        model=config["model"]["name"],
        model_kwargs={
            "embedding_dim": config["model"]["embedding_dim"],
            "random_seed": config["seed"],
        },
        training_loop="sLCWA",
        training_kwargs={
            "num_epochs": config["train"]["num_epoch"],
            "checkpoint_name" : config['model']['name'].lower()+'.pt',
            "checkpoint_frequency": 20,
        },
        optimizer=config["optimizer"]["class"],
        optimizer_kwargs={"lr": config["optimizer"]["lr"]},
        negative_sampler_kwargs={
            "num_negs_per_pos": config["train"]["num_negative"],
        },
        stopper='early',
        stopper_kwargs=dict(frequency=10, patience=3, relative_delta=0.002),
        random_seed=config["seed"],
        evaluator_kwargs={"filtered": True},
        use_testing_data=False,
        result_tracker='mlflow',
        result_tracker_kwargs=dict(
        tracking_uri='http://localhost:5000',
        experiment_name=config["experiment_name"],
    ),
    )
    
    pipeline_result.save_to_directory(config["save"]["path"])
    return pipeline_result


import yaml

In [30]:
import yaml
def load_config(config_path: str) -> dict:
    """Load a YAML config file"""

    with open(config_path, "r", encoding="utf-8") as file:
        config = yaml.safe_load(file)

    return config

rotate_best_hpo_config: dict = load_config('config/rotate-best-hpo.yaml')
rotate_best_hpo_config

{'type': 'baseline',
 'dataset': 'Hetionet',
 'experiment_name': 'RotatE best parameters',
 'model': {'name': 'RotatE', 'embedding_dim': 200},
 'optimizer': {'class': 'Adagrad', 'lr': 0.02},
 'train': {'loss_function': 'MarginRankingLoss',
  'num_epoch': 300,
  'num_negative': 1,
  'create_inverse': False},
 'save': {'path': 'results/baseline/rotate_best_hpo'},
 'seed': 84}

In [33]:
# Run model with config from checkpoint
def run_pipeline_with_checkpoint(config: dict, checkpoint_name):

    pipeline_result = pipeline(
        dataset=config["dataset"],
        dataset_kwargs={
            "random_state": config["seed"],
            "create_inverse_triples": config["train"]["create_inverse"],
        },
        model=config["model"]["name"],
        model_kwargs={
            "embedding_dim": config["model"]["embedding_dim"],
            "random_seed": config["seed"],
        },
        training_loop="sLCWA",
        training_kwargs={
            "num_epochs": config["train"]["num_epoch"],
            "checkpoint_name": checkpoint_name
        },
        optimizer=config["optimizer"]["class"],
        optimizer_kwargs={"lr": config["optimizer"]["lr"]},
        negative_sampler_kwargs={
            "num_negs_per_pos": config["train"]["num_negative"],
        },
        stopper='early',
        stopper_kwargs=dict(frequency=10, patience=3, relative_delta=0.002),
        random_seed=config["seed"],
        evaluator_kwargs={"filtered": True},
        use_testing_data=False,
        result_tracker='mlflow',
        result_tracker_kwargs=dict(
        tracking_uri='http://localhost:5000',
        experiment_name=config["experiment_name"],
    ),
    )
    
    pipeline_result.save_to_directory(config["save"]["path"])
    return pipeline_result

In [24]:
def print_metrics(metric_results):
    print("MRR: {}".format(metric_results.get_metric("inverseharmonicmeanrank")))
    print("Hits@1: {}".format(metric_results.get_metric("hits_at_1")))
    print("Hits@3: {}".format(metric_results.get_metric("hits_at_3")))
    print("Hits@10: {}".format(metric_results.get_metric("hits_at_10")))

In [ ]:
rotate_best_result = run_pipeline(rotate_best_hpo_config)

INFO:pykeen.pipeline.api:=> no training loop checkpoint file found at '/home/david/.data/pykeen/checkpoints/rotate.pt'. Creating a new file.
2024/06/19 21:42:36 INFO mlflow.tracking.fluent: Experiment with name 'RotatE best parameters' does not exist. Creating a new experiment.
INFO:pykeen.datasets.utils:Loading cached preprocessed dataset from file:///home/david/.data/pykeen/datasets/hetionet/cache/KB1g2bdd1RbUZ2kejvDzUABrtqTBI-LN
INFO:pykeen.triples.triples_factory:Loading from file:///home/david/.data/pykeen/datasets/hetionet/cache/KB1g2bdd1RbUZ2kejvDzUABrtqTBI-LN/training
INFO:pykeen.triples.triples_factory:Loading from file:///home/david/.data/pykeen/datasets/hetionet/cache/KB1g2bdd1RbUZ2kejvDzUABrtqTBI-LN/testing
INFO:pykeen.triples.triples_factory:Loading from file:///home/david/.data/pykeen/datasets/hetionet/cache/KB1g2bdd1RbUZ2kejvDzUABrtqTBI-LN/validation
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model we

In [36]:
print_metrics(rotate_best_result.metric_results)

MRR: 0.11441054940223695
Hits@1: 0.05543951648742334
Hits@3: 0.11980935027997511
Hits@10: 0.23830103990756377


In [37]:
# Make predictions on the test triples
from pykeen.evaluation import RankBasedEvaluator
evaluator = RankBasedEvaluator()
metrics_testing = evaluator.evaluate(rotate_best_result.model, mapped_triples=dataset.testing.mapped_triples)

given. This means you probably forgot to pass (at least) the training triples. Try:

    additional_filter_triples=[dataset.training.mapped_triples]

Or if you want to use the Bordes et al. (2013) approach to filtering, do:

    additional_filter_triples=[
        dataset.training.mapped_triples,
        dataset.validation.mapped_triples,
    ]

Evaluating on cuda:0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225k/225k [06:48<00:00, 551triple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 409.02s seconds


In [38]:
print_metrics(metrics_testing)

MRR: 0.07225477695465088
Hits@1: 0.026290996355879476
Hits@3: 0.0674740023109057
Hits@10: 0.1617522886854502


In [40]:
# Let's predict the top 100k predictions
from pykeen.predict import predict_all
pack_top_100k = predict_all(model=rotate_best_result.model, k=100000)
predictions_top_100k = pack_top_100k.process(factory=dataset.training)


In [42]:
predictions_top_100k.df.nlargest(n=50, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
0,13621,Compound::DB00586,4,CcSE,39799,Side Effect::C0020517,-1.425890
1,13621,Compound::DB00586,4,CcSE,40613,Side Effect::C0042109,-1.430649
2,13621,Compound::DB00586,4,CcSE,39676,Side Effect::C0018681,-1.433691
3,13621,Compound::DB00586,4,CcSE,40312,Side Effect::C0033774,-1.437372
4,13621,Compound::DB00586,4,CcSE,44330,Side Effect::C1443060,-1.439938
5,14236,Compound::DB01224,4,CcSE,40575,Side Effect::C0040822,-1.443193
6,14178,Compound::DB01165,4,CcSE,39046,Side Effect::C0000737,-1.445524
7,13621,Compound::DB00586,4,CcSE,39453,Side Effect::C0012833,-1.448069
8,13621,Compound::DB00586,4,CcSE,39419,Side Effect::C0011603,-1.450484
9,14178,Compound::DB01165,4,CcSE,40555,Side Effect::C0040264,-1.458122


In [44]:
# The top 50 results are actually semantically true - Compound - causes - Side Effect
# Let's filter the triples from the train dataset

predictions_top_100k_filtered = predictions_top_100k.filter_triples(dataset.training)

In [47]:
len(predictions_top_100k_filtered.df)

49193

In [48]:
# After filtering training triples we have 49k new predictions

In [50]:
top30_filtered  = predictions_top_100k_filtered.df.nlargest(n=30, columns="score")

In [57]:
# top30_filtered = top30_filtered.drop(['head_id', 'relation_id', 'tail_id'], axis=1)
top30_filtered

,head_label,relation_label,tail_label,score
1,Compound::DB00586,CcSE,Side Effect::C0042109,-1.430649
5,Compound::DB01224,CcSE,Side Effect::C0040822,-1.443193
8,Compound::DB00586,CcSE,Side Effect::C0011603,-1.450484
9,Compound::DB01165,CcSE,Side Effect::C0040264,-1.458122
13,Compound::DB00715,CcSE,Side Effect::C0020538,-1.467961
18,Compound::DB00586,CcSE,Side Effect::C0013404,-1.470742
21,Compound::DB01165,CcSE,Side Effect::C2047937,-1.476344
22,Compound::DB01165,CcSE,Side Effect::C0003123,-1.479222
23,Compound::DB00586,CcSE,Side Effect::C2047937,-1.480670
32,Compound::DB00586,CcSE,Side Effect::C0013604,-1.496656


In [58]:
top30_filtered.to_csv('rotate-top-30-global.csv', index=False) 

In [67]:
# Distribution by relation in the top filtered results
grouped_counts = predictions_top_100k_filtered.df.groupby(['relation_label']).size()

# Sort the grouped results by the count in descending order
sorted_grouped_counts = grouped_counts.sort_values(ascending=False)

sorted_grouped_counts

relation_label
CcSE    25968
Gr>G    12587
GiG      4208
CdG      2940
CuG      1398
GpBP     1202
GpPW      677
GcG        74
GpCC       58
GpMF       57
DaG        10
AeG         7
CbG         6
CrC         1
dtype: int64

In [72]:
# The relation Compound - treats - Disease
ctd_id = torch.as_tensor(triples_factory.relations_to_ids(["CtD"]))
test_triples = dataset.testing.mapped_triples
ctd_triples_test = test_triples[test_triples[:, 1] == ctd_id]

In [73]:
metrics_ctd_test = evaluator.evaluate(rotate_best_result.model, mapped_triples=ctd_triples_test)

given. This means you probably forgot to pass (at least) the training triples. Try:

    additional_filter_triples=[dataset.training.mapped_triples]

Or if you want to use the Bordes et al. (2013) approach to filtering, do:

    additional_filter_triples=[
        dataset.training.mapped_triples,
        dataset.validation.mapped_triples,
    ]

Evaluating on cuda:0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71.0/71.0 [00:00<00:00, 292triple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.35s seconds


In [75]:
#Let's check the evaluation only on ctd triples on test
print_metrics(metrics_ctd_test)

MRR: 0.17821967601776123
Hits@1: 0.1056338028169014
Hits@3: 0.18309859154929578
Hits@10: 0.34507042253521125


In [80]:
ctd_triples_validation = dataset.validation.mapped_triples[test_triples[:, 1] == ctd_id]
metrics_ctd_validation = evaluator.evaluate(rotate_best_result.model, mapped_triples=ctd_triples_validation)

given. This means you probably forgot to pass (at least) the training triples. Try:

    additional_filter_triples=[dataset.training.mapped_triples]

Or if you want to use the Bordes et al. (2013) approach to filtering, do:

    additional_filter_triples=[
        dataset.training.mapped_triples,
        dataset.validation.mapped_triples,
    ]

Evaluating on cuda:0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71.0/71.0 [00:00<00:00, 292triple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.25s seconds


In [81]:
#Let's check the evaluation only on ctd triples on test
print_metrics(metrics_ctd_validation)

MRR: 0.0818953663110733
Hits@1: 0.028169014084507043
Hits@3: 0.07746478873239436
Hits@10: 0.176056338028169


In [85]:
from pykeen.predict import predict_triples
pack_ctd = predict_triples(model=rotate_best_result.model, triples=ctd_triples)

df_ctd = pack_ctd.process(factory=dataset.training).df

df_ctd.nlargest(n=30, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
26,13600,Compound::DB00563,8,CtD,14826,Disease::DOID:7148,-3.618861
29,13774,Compound::DB00741,8,CtD,14791,Disease::DOID:2841,-3.702628
63,13491,Compound::DB00445,8,CtD,14788,Disease::DOID:2531,-3.750959
14,13654,Compound::DB00619,8,CtD,14788,Disease::DOID:2531,-3.768634
31,14005,Compound::DB00983,8,CtD,14791,Disease::DOID:2841,-3.779747
52,13491,Compound::DB00445,8,CtD,14774,Disease::DOID:1793,-3.780689
27,13655,Compound::DB00620,8,CtD,14799,Disease::DOID:3310,-3.794230
2,13675,Compound::DB00641,8,CtD,14802,Disease::DOID:3393,-3.802170
41,14458,Compound::DB04572,8,CtD,14768,Disease::DOID:1612,-3.823779
40,13175,Compound::DB00091,8,CtD,14834,Disease::DOID:8893,-3.847465


In [86]:
import requests

# Define a cache dictionary for disease labels
disease_cache = {}

# Get disease label from by id from  Disease Ontology
def get_disease_label(disease_id):
    if disease_id.startswith("Disease::"):
        disease_id = disease_id.split("::")[1]

    # Check if the disease ID is already in the cache
    if disease_id in disease_cache:
        return disease_cache[disease_id]
    
    url = f"https://www.ebi.ac.uk/ols/api/ontologies/doid/terms?obo_id=DOID:{disease_id.split(':')[1]}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'label' in data['_embedded']['terms'][0]:
            label = data['_embedded']['terms'][0]['label']
            # Store the label in the cache
            disease_cache[disease_id] = label
            return label
    
    # If the request fails or the label is not found, return the ID itself
    disease_cache[disease_id] = disease_id
    return disease_id

In [87]:
import pandas as pd

def create_drug_mapping(file_path):
    # Read the TSV file into a pandas DataFrame
    df = pd.read_csv(file_path, sep='\t')

    # Create a dictionary from the DataFrame
    drug_mapping = dict(zip(df['drugbankId'], df['name']))

    return drug_mapping

In [88]:
def get_drug_label(drug_id, drug_mapping):
    # Remove the prefix if present
    if drug_id.startswith("Compound::"):
        drug_id = drug_id.split("::")[1]
    
    # Return the label if present, otherwise return the ID itself
    return drug_mapping.get(drug_id, drug_id)

# Example usage
file_path = 'drug-mappings.tsv'
drug_mapping = create_drug_mapping(file_path)

In [89]:
get_drug_label("Compound::DB00445", drug_mapping)

'Epirubicin'

In [90]:
df_ctd_clean = pd.DataFrame(df_ctd)

# Replace DrugBank IDs with labels
df_ctd_clean['head_label'] = df_ctd_clean['head_label'].apply(lambda x: get_drug_label(x, drug_mapping))

# Replace Disease IDs with labels
df_ctd_clean['tail_label'] = df_ctd_clean['tail_label'].apply(get_disease_label)

# Drop the columns 'head_id', 'relation_id', 'tail_id'
df_ctd_clean = df_ctd_clean.drop(columns=['head_id', 'relation_id', 'tail_id'])

# Rename the columns for clarity
df_ctd_clean = df_ctd_clean.rename(columns={
    'head_label': 'Compound',
    'relation_label': 'Relation',
    'tail_label': 'Disease'
})

# Display the updated DataFrame
df_ctd_clean.nlargest(n=30, columns="score")

,Compound,Relation,Disease,score
26,Methotrexate,CtD,rheumatoid arthritis,-3.618861
29,Hydrocortisone,CtD,asthma,-3.702628
63,Epirubicin,CtD,hematologic cancer,-3.750959
14,Imatinib,CtD,hematologic cancer,-3.768634
31,Formoterol,CtD,asthma,-3.779747
52,Epirubicin,CtD,pancreatic cancer,-3.780689
27,Triamcinolone,CtD,atopic dermatitis,-3.794230
2,Simvastatin,CtD,coronary artery disease,-3.802170
41,Thiotepa,CtD,breast cancer,-3.823779
40,Cyclosporine,CtD,psoriasis,-3.847465


In [94]:
#Lets make prediction for arthritis
from pykeen import predict
compound_treats_arthritis = predict.predict_target(
    model=rotate_best_result.model,
    relation="CtD",
    tail="Disease::DOID:7148",
    triples_factory=dataset.training,
)

In [95]:
compound_treats_arthritis_df = compound_treats_arthritis.df
compound_treats_arthritis_df

,head_id,score,head_label
14478,14478,-3.551439,Compound::DB04868
14265,14265,-3.580540,Compound::DB01254
14019,14019,-3.618309,Compound::DB00997
13600,13600,-3.618861,Compound::DB00563
13491,13491,-3.640974,Compound::DB00445
...,...,...,...
32546,32546,-6.700233,Gene::8814
17364,17364,-6.702159,Gene::135656
15093,15093,-6.703828,Gene::100507679
28566,28566,-6.727104,Gene::64066


In [96]:
compound_treats_arthritis_df_filtered = compound_treats_arthritis_df[compound_treats_arthritis_df['head_label'].str.contains("Compound")]
compound_treats_arthritis_df_filtered

,head_id,score,head_label
14478,14478,-3.551439,Compound::DB04868
14265,14265,-3.580540,Compound::DB01254
14019,14019,-3.618309,Compound::DB00997
13600,13600,-3.618861,Compound::DB00563
13491,13491,-3.640974,Compound::DB00445
...,...,...,...
14160,14160,-5.693714,Compound::DB01145
13822,13822,-5.696471,Compound::DB00791
13208,13208,-5.795581,Compound::DB00147
14546,14546,-5.903199,Compound::DB06614


In [97]:
compound_treats_arthritis = pd.DataFrame(compound_treats_arthritis_df_filtered)
# Replace DrugBank IDs with labels
compound_treats_arthritis['head_label'] = compound_treats_arthritis['head_label'].apply(lambda x: get_drug_label(x, drug_mapping))

In [99]:
compound_treats_arthritis.nlargest(n=10, columns="score")

,head_id,score,head_label
14478,14478,-3.551439,Nilotinib
14265,14265,-3.580540,Dasatinib
14019,14019,-3.618309,Doxorubicin
13600,13600,-3.618861,Methotrexate
13491,13491,-3.640974,Epirubicin
13654,13654,-3.643833,Imatinib
14542,14542,-3.648634,Pazopanib
14654,14654,-3.657118,Ponatinib
14051,14051,-3.657606,Topotecan
14637,14637,-3.660533,Crizotinib


In [100]:
compound_treats_arthritis.to_csv('rotate-top-10-arthritis.csv', index=False)